In [1]:
from pyspark.streaming import StreamingContext

# Stateful word count

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory.  Required for stateful transformations
ssc.checkpoint("checkpoint")

rdd = sc.textFile('adj_noun_pairs.txt', 8)
rddQueue = rdd.randomSplit([1]*10, 123)
lines = ssc.queueStream(rddQueue)

def cleandata(l):
    l = l.split(" ")
    if len(l[0])!=0 and len(l[1])!=0:
        return ((l[0],l[1]),len(l[1]))
    # add the new values with the previous running count to get the new count
def updateFunc(newValues, runningCount):
    if runningCount is None :
        runningCount=0
    return max(newValues, runningCount)

#running_counts = lines.map(cleandata).reduceByKey(lambda a, b: a).updateStateByKey(updateFunc)
running_counts = lines.map(cleandata).reduceByKey(lambda a, b: a).updateStateByKey(updateFunc)
#running_counts.pprint()
counts_sorted = running_counts.transform(lambda rdd: rdd.sortBy(lambda x: x[1], False))

def printResults(rdd):
    print "Total distinct words: ", rdd.count()
    print rdd.take(5)
    #print 'refinery:', rdd.lookup('refinery')[0]

counts_sorted.foreachRDD(printResults)

ssc.start()
#ssc.awaitTermination(200)
#ssc.stop(False)
#print("Finished")

Total distinct words:  197186
[((u'delete', u'wikishouldoffersimplifieduseoftable'), [35]), ((u'advanced', u'electronic-counter-counter-measure'), [34]), ((u'delete', u'atlasshrugged/passengernumber1'), [30]), ((u'=', u'lowland_anoa_drawing_historic'), [29]), ((u'delete', u'atlasshrugged/taggartbuilding'), [29])]
Total distinct words:  341011
[((u'delete', u'whichwikishouldweuse/naminglinkingdiscussion'), [44]), ((u'delete', u'BusinessSchools/UniversityOfMichiganAnnArbor'), [44]), ((u'delete', u'wikishouldoffersimplifieduseoftable'), [35]), ((u'advanced', u'electronic-counter-counter-measure'), [34]), ((u'at-taqafah', u'wad-dimuqratiyyah/rassemblement'), [31])]
Total distinct words:  465420
[((u'delete', u'whichwikishouldweuse/naminglinkingdiscussion'), [44]), ((u'delete', u'BusinessSchools/UniversityOfMichiganAnnArbor'), [44]), ((u'stack-oriented', u'non-english-based_programming_languages'), [39]), ((u'delete', u'BusinessSchools/UniversityOfWashington'), [38]), ((u'delete', u'wikisho

Traceback (most recent call last):
  File "/home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7/SocketServer.py", line 293, in _handle_request_noblock
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
Py4JNetworkError: Error while receiving


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 56520)
----------------------------------------


    self.process_request(request, client_address)
  File "/home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7/SocketServer.py", line 655, in __init__
    self.handle()
  File "/usr/spark-2.4.4-bin-hadoop2.7/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/spark-2.4.4-bin-hadoop2.7/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/spark-2.4.4-bin-hadoop2.7/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/spark-2.4.4-bin-hadoop2.7/python/pyspark/serializers.py", line 717, in read_int
    raise EOFError
EOFError
ERROR:root:Exception 